# Laboratorio 5: Aplicación de Redes Neuronales.  Nombre: Sandra Villca Señoranis

Se implementará una red neuronal con una Capa Oculta para reconocimiento de los símbolos del alfabeto japonés Hiragana.\
Las imágenes están en el rango de valores de [0,255] en un formato csv.\
Contiene 10 Clases de 0 a 9.\
Se tiene 60000 ejemplos para entrenamiento y 10000 ejemplos para prueba.

##1. Importación de Librerías

In [1]:
# used for manipulating directory paths
import os

# Scientific and vector computation for python
import numpy as np

# Plotting library
from matplotlib import pyplot

# Optimization module in scipy
from scipy import optimize

# will be used to load MATLAB mat datafile format
# from scipy.io import loadmat

# library written for this exercise providing additional functions for assignment submission, and others
# import utils


# tells matplotlib to embed plots within the notebook
%matplotlib inline

#Libreria pandas y scikitlearn para el preprocesamiento de datasets
import pandas as pd
import sklearn as skl

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


##2. Contenido del dataset
El contenido del dataset se carga en "dataframe", el cual Panda utilizará como unidad de información.\
El conjunto de datos consta de dos archivos:

train_data.csv\
test_data.csv\
El archivo train_data.csv contiene 60.000 ejemplos y etiquetas de entrenamiento. test_data.csv contiene 10.000 ejemplos de prueba y etiquetas. Cada fila consta de 785 valores: el primer valor es la etiqueta (un número del 0 al 9) y los 784 valores restantes son los valores de los píxeles (un número del 0 al 255).

In [3]:
dataframe = pd.read_csv('/content/gdrive/MyDrive/SIS 420/Laboratorio5/Japanese_Characters/train_data.csv')
print(dataframe)
print(dataframe.describe())

       label  0  1  2  3  4    5    6   7   8  ...  774  775  776  777  778  \
0          8  0  0  0  0  0    0    0   0   0  ...  122  255   90    0    0   
1          7  0  0  0  0  0    0    0   0   0  ...    0    0    0    0    0   
2          0  0  0  0  0  0    0    0   0   0  ...    0    0    0    0    0   
3          1  0  0  0  0  0    0    0   0  32  ...  255   64    0    0    0   
4          4  0  0  0  0  0    0    0   0   0  ...    0    0    0    0    0   
...      ... .. .. .. .. ..  ...  ...  ..  ..  ...  ...  ...  ...  ...  ...   
59995      4  0  0  0  0  0    0    0   0   0  ...    0    0    0    0    0   
59996      9  0  0  0  0  0    0    0   0   0  ...    0    0    0    0    0   
59997      0  0  0  0  0  0    0    0   0   0  ...    0    0    0    0    0   
59998      4  0  0  0  0  0  103  225  53   0  ...    0    0    0    0    0   
59999      9  0  0  0  0  0    0    0   0   0  ...    0    0    0    0    0   

       779  780  781  782  783  
0        0    0   

##3.Preprocesamiento del dataset train_data.cvs

###3.1. Preprocesamiento del dataset: Separar atributos(X) de la etiqueta (y) que se va a usar para el entrenamiento
Los atributos(X) se van a cargar en la variable "dataset" y la etiqueta "y" en la variable "label".

In [4]:
dataset = dataframe.drop("label", axis=1)
label = dataframe["label"].copy()

###3.2. Preprocesamiento del dataset: Atributos con texto
En nuestro dataset no tenemos atributos NO NUMÉRICOS.

###Limpieza de valores NaN en los atributos.

In [5]:
print(dataframe.describe())

              label             0             1             2             3  \
count  60000.000000  60000.000000  60000.000000  60000.000000  60000.000000   
mean       4.500000      0.315700      0.754383      1.236933      2.163267   
std        2.872305      6.356808     10.545225     13.658704     18.474369   
min        0.000000      0.000000      0.000000      0.000000      0.000000   
25%        2.000000      0.000000      0.000000      0.000000      0.000000   
50%        4.500000      0.000000      0.000000      0.000000      0.000000   
75%        7.000000      0.000000      0.000000      0.000000      0.000000   
max        9.000000    255.000000    255.000000    255.000000    255.000000   

                  4             5             6             7            8  \
count  60000.000000  60000.000000  60000.000000  60000.000000  60000.00000   
mean       3.640333      5.816667      8.689600     12.477250     16.83380   
std       24.101083     30.263832     36.837178     44

In [6]:
# Búsqueda de valores NaN
# Itera sobre cada columna y verifica si hay valores NaN
for col in dataframe.columns:
    # Verifica si hay valores NaN en la columna actual
    if dataframe[col].isnull().sum() > 0:
        # Imprime el nombre de la columna y la cantidad de valores NaN
        print(f"Categoría '{col}' tiene {dataframe[col].isnull().sum()} valores NaN.")

###Observación: No hay valores nan

## 4. Aplicación de Redes Neuronales
###Al inicio cargamos el dataframe en data. Imprimimos los valores de 'X' e 'y' .

In [7]:
#  datos de entrenamiento almacenados en los arreglos X, y
# data = np.loadtxt("/content/gdrive/MyDrive/Colab Notebooks/machine learning/datasets/wine_preparado.csv", delimiter=',')
data = dataframe.copy()

# print(data)
#X, y = data[:, 1:], data[:,0]

#Dataframe de Pandas
X = data.iloc[:, 1:]  # Seleccionar todas las filas y las columnas de 1 a 784.
y = data.iloc[:, 0]   # Seleccionar todas las filas y solo la columna 0.

y = np.array([int(e) for e in y])
print(y.shape)
y = np.squeeze(y)

"""
y[y == 1] = 0
y[y == 2] = 1
y[y == 3] = 2

Xp= X[150:,:]
X = X[:150,:]

yp = y[150:]
y = y[:150]
"""

# print(npy)
# print(npy.shape)
# y = Y
# establecer el dígito cero en 0, en lugar del 10 asignado a este conjunto de datos
# Esto se hace debido a que el conjunto de datos se utilizó en MATLAB donde no hay índice 0
# m = y.size
print(X.shape)
print(y.shape)

#print(Xp.shape)
#print(yp.shape)


print(X)
print(y)

(60000,)
(60000, 784)
(60000,)
       0  1  2  3  4    5    6   7   8    9  ...  774  775  776  777  778  \
0      0  0  0  0  0    0    0   0   0    0  ...  122  255   90    0    0   
1      0  0  0  0  0    0    0   0   0    0  ...    0    0    0    0    0   
2      0  0  0  0  0    0    0   0   0    0  ...    0    0    0    0    0   
3      0  0  0  0  0    0    0   0  32  164  ...  255   64    0    0    0   
4      0  0  0  0  0    0    0   0   0    0  ...    0    0    0    0    0   
...   .. .. .. .. ..  ...  ...  ..  ..  ...  ...  ...  ...  ...  ...  ...   
59995  0  0  0  0  0    0    0   0   0    0  ...    0    0    0    0    0   
59996  0  0  0  0  0    0    0   0   0    0  ...    0    0    0    0    0   
59997  0  0  0  0  0    0    0   0   0    0  ...    0    0    0    0    0   
59998  0  0  0  0  0  103  225  53   0    0  ...    0    0    0    0    0   
59999  0  0  0  0  0    0    0   0   0    0  ...    0    0    0    0    0   

       779  780  781  782  783  
0        0 

###Normalización

In [8]:
def featureNormalize(X):
    X_norm = X.copy()
    mu = np.zeros(X.shape[1])
    sigma = np.zeros(X.shape[1])

    mu = np.mean(X, axis=0)
    sigma = np.std(X, axis=0)

    # Manejar el caso de desviación estándar igual a cero
    sigma[sigma == 0] = 1  # Si la desviación estándar es cero, establecerla a 1

    X_norm = (X - mu) / sigma

    return X_norm, mu, sigma

In [9]:
# llama featureNormalize con los datos cargados
X_norm, mu, sigma = featureNormalize(X)

In [10]:
print(X_norm)

              0         1         2         3         4         5         6  \
0     -0.049664 -0.071539 -0.090561 -0.117097 -0.151046 -0.192200 -0.235894   
1     -0.049664 -0.071539 -0.090561 -0.117097 -0.151046 -0.192200 -0.235894   
2     -0.049664 -0.071539 -0.090561 -0.117097 -0.151046 -0.192200 -0.235894   
3     -0.049664 -0.071539 -0.090561 -0.117097 -0.151046 -0.192200 -0.235894   
4     -0.049664 -0.071539 -0.090561 -0.117097 -0.151046 -0.192200 -0.235894   
...         ...       ...       ...       ...       ...       ...       ...   
59995 -0.049664 -0.071539 -0.090561 -0.117097 -0.151046 -0.192200 -0.235894   
59996 -0.049664 -0.071539 -0.090561 -0.117097 -0.151046 -0.192200 -0.235894   
59997 -0.049664 -0.071539 -0.090561 -0.117097 -0.151046 -0.192200 -0.235894   
59998 -0.049664 -0.071539 -0.090561 -0.117097 -0.151046  3.211231  5.872117   
59999 -0.049664 -0.071539 -0.090561 -0.117097 -0.151046 -0.192200 -0.235894   

              7         8         9  ...       774 

In [11]:
#Los datos normalizados los guardamos en X
X = X_norm

##5. Configuración de parámetros
En la Capa Oculta se ha variado los 'Valores' desde 5 hasta 100 'Valores' en los diferentes intentos de entrenamiento.\
Finalmente se establece 10 Valores para lo cual se tendra que calcular 7960 Thetas en toda la red neuronal.

In [12]:
# Configurando parametros necesario
input_layer_size  = 784  # Entrada de 784 caracteristicas

#hidden_layer_size = 100   # PRIMER INTENTO: 100 unidades ocultas
#hidden_layer_size = 50    # SEGUNDO INTENTO: 50 unidades ocultas
hidden_layer_size = 50     # TERCER INTENTO: 10 unidades ocultas

num_labels = 10          # 10 etiquetas, de 0 a 9

# carga los pesos en las variables Theta1 y Theta2
# weights = loadmat(os.path.join('/content/gdrive/MyDrive/Colab Notebooks/machine learning/data', 'ex4weights.mat'))
# weights = np.array()
pesos = {}
pesos['Theta1'] = np.random.rand(50, 785)
pesos['Theta2'] = np.random.rand(10, 51)
# print(pesos['Theta1'][:].shape)
# print(pesos['Theta2'][:].shape)

# print(weights['Theta1'][:].shape)
# print(weights['Theta2'][:].shape)

# print(weights['Theta1'][0])
# print(np.roll(weights['Theta1'][0], 1, axis=0))
# Theta1 tiene un tamaño de 25x401
# Theta2 tiene un tamañó de 10x26
# Theta1, Theta2 = weights['Theta1'], weights['Theta2']
Theta1, Theta2 = pesos['Theta1'], pesos['Theta2']
# se intercambia la ultima columa con la primera de Theta2, por cuestiones de indices que utiliza MATLAB
# print(Theta2)
# print(np.roll(Theta2, 1, axis=0))

# Theta2 = np.roll(Theta2, 1, axis=0)

# Desenrollar parámetros
print(Theta1.ravel().shape)
print(Theta2.ravel().shape)

nn_params = np.concatenate([Theta1.ravel(), Theta2.ravel()])
print(nn_params.shape)

(39250,)
(510,)
(39760,)


##6. Aplicación de la Sigmoide
Nos permite transformar una regresión que nos da un valor lineal, mediante la función de activación de la Sigmoide, en una distribución de Probabilidad entre 0 y 1.

In [14]:
def sigmoid(z):
    """
    Computes the sigmoid of z.
    """
    return 1.0 / (1.0 + np.exp(-z))


def sigmoidGradient(z):

    g = np.zeros(z.shape)

    g = sigmoid(z) * (1 - sigmoid(z))

    return g

##7. Función de Costo
En la Propagación hacia atrás: Se ajustan los valores de Theta para lo cual también se calculan las diferncias delta. Ejemplo delta_3 que es la diferencia entre el valor predicho y el grand true (y_matrix).\
En la ecuación de Costo se ha agregado un valor pequeño (epsilon) al logaritmo para evitar el logaritmo de cero np.log(a3 + epsilon) -> np.log(a3 + 0.00000001).\
Se aplica también el criterio de regularización.

In [15]:
def nnCostFunction(nn_params,
                   input_layer_size,
                   hidden_layer_size,
                   num_labels,
                   X, y, lambda_= 0.0):

    # Reshape nn_params back into the parameters Theta1 and Theta2, the weight matrices
    # for our 2 layer neural network
    Theta1 = np.reshape(nn_params[:hidden_layer_size * (input_layer_size + 1)],
                        (hidden_layer_size, (input_layer_size + 1)))

    Theta2 = np.reshape(nn_params[(hidden_layer_size * (input_layer_size + 1)):],
                        (num_labels, (hidden_layer_size + 1)))

    m = y.size

    J = 0
    Theta1_grad = np.zeros(Theta1.shape)
    Theta2_grad = np.zeros(Theta2.shape)

    a1 = np.concatenate([np.ones((m, 1)), X], axis=1)

    a2 = sigmoid(a1.dot(Theta1.T))
    a2 = np.concatenate([np.ones((a2.shape[0], 1)), a2], axis=1)

    a3 = sigmoid(a2.dot(Theta2.T))

    # print("-"*20)
    # print(y.shape)
    # print(y.reshape(-1))
    # print("-"*20)
    y_matrix = y.reshape(-1)
    # print(y.shape)
    y_matrix = np.eye(num_labels)[y_matrix]
    # print(y_matrix)

    temp1 = Theta1
    temp2 = Theta2

    # Agregar el termino de regularización

    reg_term = (lambda_ / (2 * m)) * (np.sum(np.square(temp1[:, 1:])) + np.sum(np.square(temp2[:, 1:])))

    J = (-1 / m) * np.sum((np.log(a3 + 0.00000001) * y_matrix) + np.log(1 - a3 + 0.00000001) * (1 - y_matrix)) + reg_term

    # Backpropogation

    delta_3 = a3 - y_matrix
    delta_2 = delta_3.dot(Theta2)[:, 1:] * sigmoidGradient(a1.dot(Theta1.T))

    Delta1 = delta_2.T.dot(a1)
    Delta2 = delta_3.T.dot(a2)

    # Agregar regularización al gradiente

    Theta1_grad = (1 / m) * Delta1
    Theta1_grad[:, 1:] = Theta1_grad[:, 1:] + (lambda_ / m) * Theta1[:, 1:]

    Theta2_grad = (1 / m) * Delta2
    Theta2_grad[:, 1:] = Theta2_grad[:, 1:] + (lambda_ / m) * Theta2[:, 1:]

    # ===================== Alterntate solutions =====================
    # my_final_matrix = np.zeros(a3.shape)
    # for c in np.arange(num_labels):
    #    my_final_matrix[:, c] = (np.log(a3[:, c]) * (y == c)) + (np.log(1 - a3[:, c]) * (1 - (y == c)))
    #J = (-1 / m) * np.sum(my_final_matrix)
    # ================================================================

    # ================================================================
    # Unroll gradients
    # grad = np.concatenate([Theta1_grad.ravel(order=order), Theta2_grad.ravel(order=order)])

    grad = np.concatenate([Theta1_grad.ravel(), Theta2_grad.ravel()])

    return J, grad

##Llamada a la función de costo:
Se llama a la función de costo para imprimir el error calculado para los parámetros configurados.


In [16]:
lambda_ = 0
J, _ = nnCostFunction(nn_params, input_layer_size, hidden_layer_size, num_labels, X, y, lambda_)
print('Costo en parametros: %.6f ' % J)


Costo en parametros: 82.595030 


###Evaluación de la Sigmoide para valores de z definidos.
Con ésto se verifica que está funcionando bien.

In [17]:
z = np.array([-1, -0.5, 0, 0.5, 1])
g = sigmoidGradient(z)
print('Gradiente sigmoide evaluada con [-1 -0.5 0 0.5 1]:\n  ')
print(g)

Gradiente sigmoide evaluada con [-1 -0.5 0 0.5 1]:
  
[0.19661193 0.23500371 0.25       0.23500371 0.19661193]


###8. Inicialización de los pesos
Los Theta se están inicializando en la variable W.\
1. En W, se pasa el tamaño de salida, 1 + el tamaño de entrada.
2. En W, crea números random con un tamaño (L_out, 1 + L_in) para los Thetas.

In [18]:
def randInitializeWeights(L_in, L_out, epsilon_init=0.12):
    """
    Randomly initialize the weights of a layer in a neural network.

    Parameters
    ----------
    L_in : int
        Number of incomming connections.

    L_out : int
        Number of outgoing connections.

    epsilon_init : float, optional
        Range of values which the weight can take from a uniform
        distribution.

    Returns
    -------
    W : array_like
        The weight initialiatized to random values.  Note that W should
        be set to a matrix of size(L_out, 1 + L_in) as
        the first column of W handles the "bias" terms."""


    W = np.zeros((L_out, 1 + L_in))
    W = np.random.rand(L_out, 1 + L_in) * 2 * epsilon_init - epsilon_init

    return W

###9. Inicialización de parámetros de redes neuronales
Se llama a randInitializeWeights()\
Se aplana y concatena initial_Theta1 e initial_Theta2

In [19]:
print('Inicialización de parámetros de redes neuronales...')

initial_Theta1 = randInitializeWeights(input_layer_size, hidden_layer_size)
initial_Theta2 = randInitializeWeights(hidden_layer_size, num_labels)

# Desenrrollar parametros
initial_nn_params = np.concatenate([initial_Theta1.ravel(), initial_Theta2.ravel()], axis=0)

Inicialización de parámetros de redes neuronales...


###10. Entrenamiento de la red neuronal
Descripción breve:\
options {} es un diccionario, no un hiperparámetro.\
optimize.minimize va a maximizar o minimizar la función de costo.\
En 'res' que almacena los valores de Theta se ha cambiado method a 'L-BFGS-B' (Limited-memory Broyden-Fletcher-Goldfarb-Shanno con restricciones de caja). Con este método se logra una convergencia y se obtiene los valores de Theta. Con el método 'TNC' no llega a converger.


In [20]:
#  After you have completed the assignment, change the maxiter to a larger
#  value to see how more training helps.
options= {'maxiter': 1000}


#  You should also try different values of lambda
lambda_ = 1

# Create "short hand" for the cost function to be minimized
costFunction = lambda p: nnCostFunction(p, input_layer_size,
                                        hidden_layer_size,
                                        num_labels, X, y, lambda_)

# Now, costFunction is a function that takes in only one argument
# (the neural network parameters)
res = optimize.minimize(costFunction,
                        initial_nn_params,
                        jac=True,
                        method='L-BFGS-B',
                        options=options)

# get the solution of the optimization
nn_params = res.x

# Obtain Theta1 and Theta2 back from nn_params
Theta1 = np.reshape(nn_params[:hidden_layer_size * (input_layer_size + 1)],
                    (hidden_layer_size, (input_layer_size + 1)))

Theta2 = np.reshape(nn_params[(hidden_layer_size * (input_layer_size + 1)):],
                    (num_labels, (hidden_layer_size + 1)))

###11. Precisión con los datos de entrenamiento

In [21]:
def predict(Theta1, Theta2, X):
    """
    Predict the label of an input given a trained neural network
    Outputs the predicted label of X given the trained weights of a neural
    network(Theta1, Theta2)
    """
    # Useful values
    m = X.shape[0]
    num_labels = Theta2.shape[0]

    # You need to return the following variables correctly
    p = np.zeros(m)
    h1 = sigmoid(np.dot(np.concatenate([np.ones((m, 1)), X], axis=1), Theta1.T))
    h2 = sigmoid(np.dot(np.concatenate([np.ones((m, 1)), h1], axis=1), Theta2.T))
    p = np.argmax(h2, axis=1)
    return p

In [22]:
pred = predict(Theta1, Theta2, X)
print(pred)
print('Training Set Accuracy: %f' % (np.mean(pred == y) * 100))

[8 7 0 ... 0 4 9]
Training Set Accuracy: 99.941667


##12. Preprocesamiento del dataset test_data.cvs para hacer Predicciones con los datos de prueba

In [23]:
dataframe = pd.read_csv('/content/gdrive/MyDrive/SIS 420/Laboratorio5/Japanese_Characters/test_data.csv')
print(dataframe)
print(dataframe.describe())

      label  0   1   2  3  4   5    6    7    8  ...  774  775  776  777  778  \
0         2  0  97  35  0  0   0    0    0    0  ...    0    0    0    0    0   
1         9  0   0   0  0  0  24   72    0    0  ...  251  149    1    0    0   
2         3  0   0   0  0  0   0    0    0    0  ...    0    0    0    0    0   
3         8  0   0   0  0  0  17  138  195  253  ...    0    0    0    0    0   
4         3  0   0   0  0  0   0    0    0    0  ...    0    0    0    0    0   
...     ... ..  ..  .. .. ..  ..  ...  ...  ...  ...  ...  ...  ...  ...  ...   
9995      4  0   0   0  0  0   0    0    0    0  ...    0    0    0    0    0   
9996      0  0   0   0  0  0   0    0    0    0  ...    0    0    0    0    0   
9997      9  0   0   0  0  0   0    0    0  155  ...   84   59    0    0    0   
9998      4  0   0   0  0  0   0    0    0    0  ...    0    0    0    0    0   
9999      2  0   0   0  0  0   0    0    0    0  ...    0    0    0    0    0   

      779  780  781  782  7

###Separar atributos(X) de la etiqueta (y) que se va a usar para la predicción

In [24]:
dataset = dataframe.drop("label", axis=1)
label = dataframe["label"].copy()

##Cargamos el dataframe en data.

In [25]:
data = dataframe.copy()
#Dataframe de Pandas
Xp = data.iloc[:, 1:]  # Seleccionar todas las filas y las columnas de 1 a 784.
yp = data.iloc[:, 0]   # Seleccionar todas las filas y solo la columna 0.

yp = np.array([int(e) for e in yp])
print(yp.shape)
yp = np.squeeze(yp)

"""
y[y == 1] = 0
y[y == 2] = 1
y[y == 3] = 2
"""
print(Xp.shape)
print(yp.shape)

#print(X)
#print(y)

(10000,)
(10000, 784)
(10000,)


###13. Predicción y Precisión

In [26]:
pred = predict(Theta1, Theta2, Xp)
print(pred)
print('Training Set Accuracy: %f' % (np.mean(pred == yp) * 100))


[0 9 3 ... 0 7 7]
Training Set Accuracy: 58.840000


<ipython-input-14-fcf1f7fde265>:5: RuntimeWarning: overflow encountered in exp
  return 1.0 / (1.0 + np.exp(-z))


###14. Observaciones
Se ha realizado 8 entrenamientos con diferentes configuraciones de los parámetros.\
Se observa que cuando se aumenta el tamaño de la Capa Oculta la convergencia se hace más lenta dado que se tiene que determinar miles de parámetros Theta.\
Los resultados de algunos de los entrenamientos son:\
PRIMER ENTRENAMIENTO:\
Tamaño de la Capa Oculta: 10\
Method = 'TNC'\
No hay convergencia\
SEGUNDO ENTRENAMIENTO:\
Tamaño de la Capa Oculta: 10, maxiter = 100\
Method = 'L-BFGS-B'\
Costo en parámetros J: 28.58\
Precisión de entrenamiento: 85.76%\
Precisión de prueba: 51.98%\
CUARTO ENTRENAMIENTO:\
Tamaño de la Capa Oculta: 10, maxiter = 1000\
Method = 'L-BFGS-B'\
Costo en parámetros J: 26.99\
Precisión de entrenamiento: 87.50%\
Precisión de prueba: 54.50%\
OCTAVO ENTRENAMIENTO:\
Tamaño de la Capa Oculta: 50, maxiter = 1000\
Method = 'L-BFGS-B'\
Costo en parámetros J: 82.59\
Precisión de entrenamiento: 99.94%\
Precisión de prueba: 58.84%\



###15. Conclusión
En el octavo entrenamiento se obtiene el mejor valor de precisión (58.84) para el conjunto de prueba. No es una precisión óptima pero en éste caso se acepta.\
Se podría subir la precisión del conjunto de prueba aumentando el tamaño de la Capa Oculta, sin embargo está la limitación de la capacidad de procesamiento de nuestra PC dado que lleva mucho tiempo en converger y algunas veces hay desbordamiento de la memoria.\
En el procesamiento con Pytorch se puede observar que si es posible obtener una precisión alta en los conjuntos de prueba.
